### 1.Tool cration

In [43]:
from langchain_core.tools import tool
import requests
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.messages import HumanMessage


In [44]:
@tool
def get_converstation_factor(base_currency:str, target_currency:str) -> float:
    """This function fetches the currency conversion factor between a given base currency and a target currency"""
    
    url=f"https://v6.exchangerate-api.com/v6/97899f18859aee6a0dd3d656/pair/{base_currency}/{target_currency}"
    
    response=requests.get(url)
    
    return response.json()

In [45]:
get_converstation_factor.invoke({'base_currency':"USD",'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1771200002,
 'time_last_update_utc': 'Mon, 16 Feb 2026 00:00:02 +0000',
 'time_next_update_unix': 1771286402,
 'time_next_update_utc': 'Tue, 17 Feb 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.6677}

In [46]:
@tool
def convert(base_currency:int,conversion_rate: float) -> float:
    """given a currency rate this function calculates the target currency value from a given base currency value"""
    
    return base_currency*conversion_rate

In [47]:
convert.invoke({'base_currency':10,'conversion_rate':90.6677})

906.6769999999999

### 2.binding tool

In [48]:
# Load environment variables from .env
load_dotenv()

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",                   # correct parameter
    google_api_key=os.environ["GOOGLE_API_KEY"],  # correct parameter
    temperature=0
)

In [49]:
llm_with_tool=llm.bind_tools([get_converstation_factor,convert])

In [50]:
messages = [
    HumanMessage(
        content="The exchange rate for USD to INR is 83.5. "
                "Use the get_converstation_factor tool to verify this, "
                "and simultaneously use the convert tool to change 10 USD to INR."
    )
]

### so here we learn about promting

In [51]:
"""“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”"""

'“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”'

In [52]:
"""messages=[HumanMessage('what is conversion factor between USD and INR , and based on that can you conver 10 usd to inr')]"""

"messages=[HumanMessage('what is conversion factor between USD and INR , and based on that can you conver 10 usd to inr')]"

In [53]:
"""“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”"""

'“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”'

In [54]:
"""messages = [
    HumanMessage(
        content="The exchange rate for USD to INR is 83.5. "
                "Use the get_converstation_factor tool to verify this, "
                "and simultaneously use the convert tool to change 10 USD to INR."
    )
]"""

'messages = [\n    HumanMessage(\n        content="The exchange rate for USD to INR is 83.5. "\n                "Use the get_converstation_factor tool to verify this, "\n                "and simultaneously use the convert tool to change 10 USD to INR."\n    )\n]'

In [55]:
ai_message=llm_with_tool.invoke(messages)

In [56]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"conversion_rate": 83.5, "base_currency": 10}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--019c651d-fb2d-78f0-a41f-b445524e6790-0', tool_calls=[{'name': 'get_converstation_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '62fef335-f5e8-4dc9-9c0d-4b56526bfa0a', 'type': 'tool_call'}, {'name': 'convert', 'args': {'conversion_rate': 83.5, 'base_currency': 10}, 'id': '53ffcffe-e651-4365-bdef-2e0ca1519dfe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 179, 'output_tokens': 188, 'total_tokens': 367, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 135}})

In [57]:
ai_message.tool_calls

[{'name': 'get_converstation_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '62fef335-f5e8-4dc9-9c0d-4b56526bfa0a',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'conversion_rate': 83.5, 'base_currency': 10},
  'id': '53ffcffe-e651-4365-bdef-2e0ca1519dfe',
  'type': 'tool_call'}]